In [1]:
import numpy as np 
import pandas as pd

In [2]:
df = pd.read_csv('profit_full.csv')
print(df)

       gold  estate    cryto  stock class risk  profit
0     14.96   41.76  1024.07  73.17     B    Y  463.26
1     25.18   62.96  1020.04  59.08     B    N  444.37
2      5.11   39.40  1012.16  92.14     B    N  488.56
3     20.86   57.32  1010.24  76.64     B    Y  446.48
4     10.82   37.50  1009.23  96.62     C    N  473.90
...     ...     ...      ...    ...   ...  ...     ...
9563  16.65   49.69  1014.01  91.00     C    N  460.03
9564  13.19   39.18  1023.67  66.78     A    Y  469.62
9565  31.32   74.33  1012.92  36.48     A    N  429.57
9566  24.48   69.45  1013.86  62.39     A    N  435.74
9567  21.60   62.52  1017.23  67.87     B    N  453.28

[9568 rows x 7 columns]


In [3]:
df.info()
#xem thong tin

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9568 entries, 0 to 9567
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   gold    9568 non-null   float64
 1   estate  9568 non-null   float64
 2   cryto   9568 non-null   float64
 3   stock   9568 non-null   float64
 4   class   9568 non-null   object 
 5   risk    9568 non-null   object 
 6   profit  9568 non-null   float64
dtypes: float64(5), object(2)
memory usage: 523.4+ KB


In [ ]:
print(df.describe(include='all'))
#xem tổng quan dữ liệu trong DataFrame
#cho mọi loại cột (cả số và chữ).

               gold       estate        cryto        stock class  risk  \
count   9568.000000  9568.000000  9568.000000  9568.000000  9568  9568   
unique          NaN          NaN          NaN          NaN     3     2   
top             NaN          NaN          NaN          NaN     C     N   
freq            NaN          NaN          NaN          NaN  4433  4851   
mean      19.651231    54.305804  1013.259078    73.308978   NaN   NaN   
std        7.452473    12.707893     5.938784    14.600269   NaN   NaN   
min        1.810000    25.360000   992.890000    25.560000   NaN   NaN   
25%       13.510000    41.740000  1009.100000    63.327500   NaN   NaN   
50%       20.345000    52.080000  1012.940000    74.975000   NaN   NaN   
75%       25.720000    66.540000  1017.260000    84.830000   NaN   NaN   
max       37.110000    81.560000  1033.300000   100.160000   NaN   NaN   

             profit  
count   9568.000000  
unique          NaN  
top             NaN  
freq            NaN  
m

In [ ]:
x = df.iloc[:,:-1] #chọn tất cả các cột trừ cột cuối (biến độc lập)
y = df.iloc[:,-1] #chọn cột cuối cùng (biến phụ thuộc)

from sklearn.model_selection import train_test_split #chia dữ liệu ngẫu nhiên
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42) 

from sklearn.pipeline import Pipeline # Thiết lập pipeline

#Các bước tiền xử lý khác nhau
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, PowerTransformer, PolynomialFeatures
from sklearn.linear_model import LinearRegression

# import lib
from sklearn.compose import make_column_transformer
# mã hóa giả của các tính năng phân loại
from sklearn.preprocessing import OneHotEncoder

#tạo một bộ chuyển đổi cho Sex và Embarked với OHE
column_trans = make_column_transformer(
    (OneHotEncoder(), ['class', 'risk']),
    (Normalizer(), ['cryto', 'stock']),
    remainder='passthrough'
)

pipe = Pipeline(steps=[
    ('columntransformer', column_trans),
    ('std_scaler', StandardScaler()), #Chuẩn hóa toàn bộ dữ liệu
    ('poly_fe', PolynomialFeatures(degree=4)), #Tạo thêm các đặc trưng đa thức bậc 4
    ('power_fe', PowerTransformer(copy=True, method='yeo-johnson', standardize=True)), #Giảm lệch phân phối bằng PowerTransformer
    ('mm_scaler', MinMaxScaler()), #Đưa dữ liệu về khoảng [0,1]
    ('mlr_model', LinearRegression()) #Huấn luyện mô hình Linear Regression
])

pipe.fit(X_train, y_train)

print('Training set score: ' + str(pipe.score(X_train, y_train))) #độ chính xác (R²) trên tập huấn luyện.
print('Test set score: ' + str(pipe.score(X_test, y_test))) #độ chính xác trên tập kiểm thử — dùng để đánh giá khả năng khái quát hóa của mô hình.

#Nếu hai giá trị gần nhau: mô hình tổng quát tốt.
#Nếu chênh lệch lớn: có thể bị overfitting.


Training set score: 0.945216846700043
Test set score: 0.9156519701688749


In [11]:
print(pipe.get_params().keys())

dict_keys(['memory', 'steps', 'transform_input', 'verbose', 'columntransformer', 'std_scaler', 'poly_fe', 'power_fe', 'mm_scaler', 'mlr_model', 'columntransformer__force_int_remainder_cols', 'columntransformer__n_jobs', 'columntransformer__remainder', 'columntransformer__sparse_threshold', 'columntransformer__transformer_weights', 'columntransformer__transformers', 'columntransformer__verbose', 'columntransformer__verbose_feature_names_out', 'columntransformer__onehotencoder', 'columntransformer__normalizer', 'columntransformer__onehotencoder__categories', 'columntransformer__onehotencoder__drop', 'columntransformer__onehotencoder__dtype', 'columntransformer__onehotencoder__feature_name_combiner', 'columntransformer__onehotencoder__handle_unknown', 'columntransformer__onehotencoder__max_categories', 'columntransformer__onehotencoder__min_frequency', 'columntransformer__onehotencoder__sparse_output', 'columntransformer__normalizer__copy', 'columntransformer__normalizer__norm', 'std_scal

In [10]:
print(pipe.get_params())

{'memory': None, 'steps': [('columntransformer', ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder', OneHotEncoder(),
                                 ['class', 'risk']),
                                ('normalizer', Normalizer(),
                                 ['cryto', 'stock'])])), ('std_scaler', StandardScaler()), ('poly_fe', PolynomialFeatures(degree=4)), ('power_fe', PowerTransformer()), ('mm_scaler', MinMaxScaler()), ('mlr_model', LinearRegression())], 'transform_input': None, 'verbose': False, 'columntransformer': ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder', OneHotEncoder(),
                                 ['class', 'risk']),
                                ('normalizer', Normalizer(),
                                 ['cryto', 'stock'])]), 'std_scaler': StandardScaler(), 'poly_fe': PolynomialFeatures(degree=4), 'power_fe': PowerTransformer(), 'mm_scaler': MinMaxScaler(), 'mlr_model': L

In [14]:
print(pipe.named_steps['mlr_model'].intercept_)
print(pipe.named_steps['mlr_model'].coef_)

2818068469.4899735
[-2.32129945e+03  1.75402977e+05  3.04771312e+05 -5.87350707e+05
 -3.10870227e+05  3.12902608e+05  2.79073584e+09  2.05056368e+09
 -9.73523753e+05 -7.70314942e+06  2.75336226e+05 -4.82155875e+05
  1.26477159e+05  4.23378824e+06 -4.21738842e+06 -7.91293846e+07
 -1.18890191e+09 -4.10226481e+07  2.91882717e+06  3.39091833e+05
 -3.20295248e+04 -2.09074839e+06  2.07044094e+06  1.48034812e+09
  3.65320011e+09 -7.41159334e+07 -1.46972834e+07 -5.87412176e+05
 -3.15290431e+06  3.16473269e+06  2.57595872e+09  7.82169860e+08
  1.23893863e+07  3.40945528e+06  3.45115298e+05 -3.44397088e+05
  6.90214745e+08  1.11939780e+09  6.66450369e+06  1.69603335e+07
  3.46832601e+05 -6.90223508e+08 -1.11940195e+09 -6.66709010e+06
 -1.69280141e+07  6.97967625e+05  1.83882049e+06  5.74476781e+03
 -3.67435122e+04  7.07964432e+05  5.11533030e+03 -1.70614035e+04
  5.00827876e+02 -1.07272402e+02  2.60924244e+03  2.39845981e+05
  4.79344381e+04 -5.01012553e+05  2.72526147e+06 -2.71250343e+06
  1.36

In [15]:
y_pred = pipe.predict(X_test)
np.set_printoptions(precision=2)
print(y_pred)


[453.73 437.89 434.56 ... 456.44 471.44 448.96]


In [17]:
import numpy as np
from sklearn import metrics

y_test_pred = pipe.predict(X_test)

print('Score or R-Squared:', pipe.score(X_test, y_test))

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_test_pred))

print('Root Mean Squared Error:',
      np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

# Calculate MAPE manually
mape = np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100
print('Mean Absolute Percentage Error:', mape)


Score or R-Squared: 0.9156519701688749
Mean Absolute Error: 3.3300786198979635
Root Mean Squared Error: 4.951604587036658
Mean Absolute Percentage Error: 0.7343195894241337


In [19]:
from sklearn.model_selection import RandomizedSearchCV

param_space = {
    'mlr_model__copy_X': [True, False],
    'mlr_model__fit_intercept': [True, False],
    'mlr_model__n_jobs': [1, 5]
}

random_search = RandomizedSearchCV(pipe, param_space, n_iter=2, cv=5)
random_search.fit(X_train, y_train)

print(f"Best Hyperparameters: {random_search.best_params_}")
print(f"Best Score: {random_search.best_score_}")


Best Hyperparameters: {'mlr_model__n_jobs': 5, 'mlr_model__fit_intercept': False, 'mlr_model__copy_X': False}
Best Score: 0.9197073384049015
